## Carga de datos y limpieza.

Primero realizaremos la limpieza de datos para poder trabajar con ellos más cómodamente. Los archivos que vamos a usar y limpiar son los siguientes:
- Link que contiene los **pilotos ganadores** del campeonato de F1.
- Link con los **equipos ganadores** del campeonato de F1.
- Datase de Kaggle con todas las carreras disputadas en la F1.

In [9]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

Primero cargaremos los datos de los pilotos y equipos campeones para juntarlos en un solo dataframe.

In [10]:
# Equipos ganadores del campeonato. Creamos este df para saber cual es el equipo que
# ganó el mundial y juntarlo con el de pilotos ganadores.
url = "https://es.wikipedia.org/wiki/Campeonato_Mundial_de_Constructores_de_F%C3%B3rmula_1"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find_all('table', {'class': 'wikitable sortable'})[0]
rows = table.find_all('tr')
headers = [th.get_text(strip=True) for th in rows[0].find_all('th')]

data = []
for row in rows[1:]:
    cols = row.find_all('td')
    if len(cols) == 12:
        cols = [col.get_text(strip=True) for col in cols]
        data.append(cols)

df_equipos_ganadores = pd.DataFrame(data, columns=headers)
df_equipos_ganadores.head(5)

,Año,Constructor,Motor,Chasis,Neu.,Pilotos[t 1]​,PP.,Vic.,Pod.,VR.,Pun.[t 2]​,Mar.
0,1958,Vanwall,Vanwall,VW 5,D,Stirling MossTony Brooks,5,6,9,3,48,8
1,1959,Cooper,Climax,T51,D,Jack Brabham*Stirling MossBruce McLaren,5,5,13,5,40,8
2,1960,Cooper,Climax,T51T53,D,Jack Brabham*Bruce McLaren,4,6,14,5,48,14
3,1961,Ferrari,Ferrari,156,D,Phil Hill*Wolfgang von Trips,6,5,14,5,45,10
4,1962,BRM,BRM,P57,D,Graham Hill*,1,4,8,3,42,6


In [12]:
# Pilotos ganadores del campeonato.
url = "https://en.wikipedia.org/wiki/List_of_Formula_One_World_Drivers%27_Champions"

response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")
tabla = soup.find("table", {"class": "wikitable sortable"})
filas = tabla.find_all("tr")
data = []

for fila in filas[1:]:  # Ignora la primera fila que es el encabezado
    cols = fila.find_all("td")
    
    if len(cols) == 16:
        year = cols[0].text.strip()
        driver = cols[1].text.strip()
        age = cols[2].text.strip()
        team = cols[3].text.strip()
        engine = cols[4].text.strip()
        tyres = cols[5].text.strip()
        pole = cols[6].text.strip()
        wins = cols[7].text.strip()
        podiums = cols[8].text.strip()
        fastest_laps = cols[9].text.strip()
        points = cols[10].text.strip()
        percent_points = cols[11].text.strip()
        clinched = cols[12].text.strip()
        rounds_remaining = cols[13].text.strip()
        margin = cols[14].text.strip()
        percent_margin = cols[15].text.strip()
        
        data.append([year, driver, age, team, engine, tyres, pole, wins, 
                     podiums, fastest_laps, points, percent_points, clinched, 
                     rounds_remaining, margin, percent_margin])

df_temporadas_f1 = pd.DataFrame(data, columns=[
    "year", "Driver", "Age", "Team", "Engine", "Tyres", "Pole", 
    "Wins", "Podiums", "Fastest Laps", "Points", "% Points", "Clinched", 
    "# of Rounds Remaining", "Margin", "% Margin"
])

# Eliminar los corchetes y contenido de las columnas que tienen corchetes para limpiar los datos.
for column in df_temporadas_f1.columns:
    for index in range(len(df_temporadas_f1)):
        df_temporadas_f1.at[index, column] = re.sub(r'\[.*?\]', '', str(df_temporadas_f1.at[index, column]))

# Convertir los datos de la variable "Tyres" en un solo valor para poder analizarlo mejor.
df_temporadas_f1["year"] = pd.to_numeric(df_temporadas_f1["year"])
df_temporadas_f1.loc[df_temporadas_f1["Tyres"] == "F P", "Tyres"] = "F"
df_temporadas_f1.loc[df_temporadas_f1["Tyres"] == "M G", "Tyres"] = "G"

# Convertir las variables númericas en tipo "int" y "float".
columnas_cambio_tipo = ["Age","Pole","Wins","Podiums","Fastest Laps","# of Rounds Remaining"]
df_temporadas_f1[columnas_cambio_tipo] = df_temporadas_f1[columnas_cambio_tipo].astype(int)
df_temporadas_f1["Points"] = df_temporadas_f1["Points"].astype(float)

df_temporadas_f1.head(5)

,year,Driver,Age,Team,Engine,Tyres,Pole,Wins,Podiums,Fastest Laps,Points,% Points,Clinched,# of Rounds Remaining,Margin,% Margin
0,1950,Giuseppe Farina,44,Alfa Romeo,Alfa Romeo,P,2,3,3,3,30.0,83.333 (47.619),Round 7 of 7,0,3,10.000
1,1951,Juan Manuel Fangio,40,Alfa Romeo,Alfa Romeo,P,4,3,5,5,31.0,86.111 (51.389),Round 8 of 8,0,6,19.355
2,1952,Alberto Ascari,34,Ferrari,Ferrari,F,5,6,6,6,36.0,100.000 (74.306),Round 6 of 8,2,12,33.333
3,1953,Alberto Ascari,35,Ferrari,Ferrari,P,6,5,5,4,34.5,95.833 (57.407),Round 8 of 9,1,6.5,18.841
4,1954,Juan Manuel Fangio,43,Maserati,Maserati,P,5,6,7,3,42.0,93.333 (70.547),Round 7 of 9,2,16.857,40.136


Una vez tenemos los dos dataframes cargados correctamente, los juntamos para tener los datos necesarios en uno solo y poder manipularlo de fácilmente a la hora de realizar el análisis. Tan solo cogeremos la columna `Constructor` para juntarla con los datos de los pilotos.
Como el premio a equipo ganador del campeonato se comenzó a dar a partir de 1958, desde el año 1950 hasta 

In [15]:
# Añadir filas por encima del df de equipos ganadores para poder juntarlo con el otro y cuadre el número de filas.
diferencia = 75 - len(df_equipos_ganadores)

# Si faltan filas, añadir filas vacías solo al principio.
if diferencia > 0:
    filas_extra = pd.DataFrame( index=range(diferencia), columns=df_equipos_ganadores.columns)
    df_equipos_ganadores = pd.concat([filas_extra, df_equipos_ganadores], ignore_index=True)

df_temporadas_f1["Team_winner"] = df_equipos_ganadores["Constructor"]
# Sustituir los nulos por "Not awarded".
df_temporadas_f1["Team_winner"] = df_temporadas_f1["Team_winner"].fillna("Not awarded")
df_temporadas_f1.to_csv("df_temporadas_f1.csv", index=False)

df_temporadas_f1.head(5)

,year,Driver,Age,Team,Engine,Tyres,Pole,Wins,Podiums,Fastest Laps,Points,% Points,Clinched,# of Rounds Remaining,Margin,% Margin,Team_winner
0,1950,Giuseppe Farina,44,Alfa Romeo,Alfa Romeo,P,2,3,3,3,30.0,83.333 (47.619),Round 7 of 7,0,3,10.000,Not awarded
1,1951,Juan Manuel Fangio,40,Alfa Romeo,Alfa Romeo,P,4,3,5,5,31.0,86.111 (51.389),Round 8 of 8,0,6,19.355,Not awarded
2,1952,Alberto Ascari,34,Ferrari,Ferrari,F,5,6,6,6,36.0,100.000 (74.306),Round 6 of 8,2,12,33.333,Not awarded
3,1953,Alberto Ascari,35,Ferrari,Ferrari,P,6,5,5,4,34.5,95.833 (57.407),Round 8 of 9,1,6.5,18.841,Not awarded
4,1954,Juan Manuel Fangio,43,Maserati,Maserati,P,5,6,7,3,42.0,93.333 (70.547),Round 7 of 9,2,16.857,40.136,Not awarded


Una vez tenemos el dataframe de las temporadas con los datos necesarios, realizamos la carga y limpieza de los datos para el dataframe de todas las `carreras`. 
El proceso será distinto al realizado anteriormente ya que los datos de las carreras se encuentran en **Kaggle**.

In [17]:
# Extracción de los datos para crear el df.
circuits = pd.read_csv("../data/f1_data/circuits.csv")
constructors = pd.read_csv("../data/f1_data/constructors.csv")
constructor_results = pd.read_csv("../data/f1_data/constructor_results.csv")
constructor_standings = pd.read_csv("../data/f1_data/constructor_standings.csv")
drivers = pd.read_csv("../data/f1_data/drivers.csv")
driver_standings = pd.read_csv("../data/f1_data/driver_standings.csv")
lap_times = pd.read_csv("../data/f1_data/lap_times.csv")
pit_stops = pd.read_csv("../data/f1_data/pit_stops.csv")
qualifying = pd.read_csv("../data/f1_data/qualifying.csv")
races = pd.read_csv("../data/f1_data/races.csv")
results = pd.read_csv("../data/f1_data/results.csv")
seasons = pd.read_csv("../data/f1_data/seasons.csv")
status = pd.read_csv("../data/f1_data/status.csv")

df_combinado = pd.merge(results, races, on='raceId', how='left')

df_combinado = pd.merge(df_combinado, drivers, on='driverId', how='left')

if 'constructorId' in constructors.columns:
    df_combinado = pd.merge(df_combinado, constructors, on='constructorId', how='left')

if 'constructorId' in constructor_results.columns:
    df_combinado = pd.merge(df_combinado, constructor_results, on=['raceId', 'constructorId'], how='left')

if 'constructorId' in constructor_standings.columns:
    df_combinado = pd.merge(df_combinado, constructor_standings, on=['raceId', 'constructorId'], how='left')

if 'raceId' in qualifying.columns and 'driverId' in qualifying.columns:
    df_combinado = pd.merge(df_combinado, qualifying, on=['raceId', 'driverId'], how='left', suffixes=('', '_qualifying'))

df_combinado = df_combinado.loc[:, ~df_combinado.columns.str.endswith('_qualifying')]

if 'raceId' in driver_standings.columns and 'driverId' in driver_standings.columns:
    df_combinado = pd.merge(df_combinado, driver_standings, on=['raceId', 'driverId'], how='left', suffixes=('', '_driver_standings'))

# Eliminar las columnas duplicadas si ya existen en el DataFrame
df_combinado = df_combinado.loc[:, ~df_combinado.columns.str.endswith('_driver_standings')]

if 'year' in seasons.columns:
    df_combinado = pd.merge(df_combinado, seasons, on='year', how='left', suffixes=('', '_seasons'))

    df_combinado = df_combinado.loc[:, ~df_combinado.columns.str.endswith('_seasons')]
    
if 'statusId' in status.columns:
    df_combinado = pd.merge(df_combinado, status, on='statusId', how='left')

# Eliminación de duplicados.
df_combinado = df_combinado.drop_duplicates()
pd.set_option("display.max_columns", None)

# Eliminar colunmas que no nos interesan.
df_combinado.drop(columns=["url_x","url_y","url","quali_date","quali_time","sprint_date","sprint_time",
                          "name_y","surname","fp1_date","fp1_time","fp2_date","fp2_time",
                          "fp3_date","fp3_time","milliseconds","time_y","positionText_x","number_y",
                          "positionText_y","driverId","qualifyId","number","statusId","resultId","constructorId",
                          "positionText","rank","driverStandingsId","status_x","constructorResultsId",
                          "constructorStandingsId","position","circuitId"], inplace = True)

# Renombrar columnas para entenderlas.
df_combinado.rename(columns={"nationality_y":"team_nation","nationality_x":"driver_nation","constructorRef":"team",
                            "date":"race_date","name_x":"grand_prix_name","time_x":"driver_race_time",
                            "position_x":"driver_race_position","number_x":"car_number","points_x":"driver_points",
                            "wins":"team_wins","points_y":"team_points","position_y":"team_position",
                            "status_y":"driver_race_status","points":"total_points"}, inplace=True)

df_combinado.head()

,raceId,car_number,grid,driver_race_position,positionOrder,driver_points,laps,driver_race_time,fastestLap,fastestLapTime,fastestLapSpeed,year,round,grand_prix_name,race_date,driverRef,code,forename,dob,driver_nation,team,team_nation,team_points,total_points,team_position,team_wins,q1,q2,q3,driver_race_status
0,18,22,1,1,1,10.0,58,1:34:50.616,39,1:27.452,218.300,2008,1,Australian Grand Prix,2008-03-16,hamilton,HAM,Lewis,1985-01-07,British,mclaren,British,14.0,14.0,1.0,1.0,1:26.572,1:25.187,1:26.714,Finished
1,18,3,5,2,2,8.0,58,+5.478,41,1:27.739,217.586,2008,1,Australian Grand Prix,2008-03-16,heidfeld,HEI,Nick,1977-05-10,German,bmw_sauber,German,8.0,8.0,3.0,0.0,1:25.960,1:25.518,1:27.236,Finished
2,18,7,7,3,3,6.0,58,+8.163,41,1:28.090,216.719,2008,1,Australian Grand Prix,2008-03-16,rosberg,ROS,Nico,1985-06-27,German,williams,British,9.0,9.0,2.0,0.0,1:26.295,1:26.059,1:28.687,Finished
3,18,5,11,4,4,5.0,58,+17.181,58,1:28.603,215.464,2008,1,Australian Grand Prix,2008-03-16,alonso,ALO,Fernando,1981-07-29,Spanish,renault,French,5.0,5.0,4.0,0.0,1:26.907,1:26.188,\N,Finished
4,18,23,3,5,5,4.0,58,+18.014,43,1:27.418,218.385,2008,1,Australian Grand Prix,2008-03-16,kovalainen,KOV,Heikki,1981-10-19,Finnish,mclaren,British,14.0,14.0,1.0,1.0,1:25.664,1:25.452,1:27.079,Finished
